### Run MMseqs2 To Detect Sequences Identical To Antigens
#### This bash script creates databases for the query (non-antigens) and target (antigens) sequences, runs the alignment search to find matches, and converts the results to a tabular format. The search is conducted using minimum sequence indentity of 30% and a coverage of 80%. The output Result.m8 file consists of columns mentioned in --format-output

In [1]:
%%bash

mkdir tmp
mmseqs easy-search  ../../SEQUENCES-COLLECTION/non_antigenic_sequences.fasta ../../SEQUENCES-COLLECTION/antigenic_sequences.fasta Result.m8 tmp --min-seq-id 0.3 -c 0.8 --cov-mode 2 -s 7.0 --format-output "qheader,query,target,pident,tlen,alnlen,qcov,evalue,bits,mismatch" --format-mode 4 -v 1

#### Import Packages

In [2]:
import pandas as pd
import gzip
from Bio import SeqIO

### Read result.m8 file.
#### qheader: Header of the query sequence
#### query: Query sequence identifier
#### target: Target sequence identifier
#### pident: Percentage of identical matches
#### tlen: Target sequence length
#### alnlen: Alignment length (number of aligned columns)
#### qcov: Fraction of query sequence covered by alignment
#### evalue: E value
#### bits: Bit score
#### mismatches: Number of mismatches

In [3]:
df = pd.read_csv("Result.m8", sep='\t')
df

,qheader,query,target,pident,tlen,alnlen,qcov,evalue,bits,mismatch
0,sp|A0A089QRB9|MSL3_MYCTU Mycolipanoate synthas...,A0A089QRB9,P9WQE9,65.1,2126,2090,0.994,0.000000e+00,2677,723
1,sp|O51312|ENO_BORBU Enolase OS=Borreliella bur...,O51312,Q9Z7A6,56.2,428,423,0.977,2.676000e-145,458,182
2,sp|O51312|ENO_BORBU Enolase OS=Borreliella bur...,O51312,O84591,55.7,424,423,0.977,2.409000e-144,456,184
3,sp|P07821|FHUC_ECOLI Iron(3+)-hydroxamate impo...,P07821,P9WQJ9,30.2,859,215,0.811,2.071000e-21,90,147
4,sp|P0A850|TIG_ECOLI Trigger factor OS=Escheric...,P0A850,Q83DJ3,41.4,442,432,0.988,2.596000e-102,334,250
...,...,...,...,...,...,...,...,...,...,...
3989,sp|O07226|Y299_MYCTU Toxin Rv0299 OS=Mycobacte...,O07226,O07226,100.0,100,100,1.000,2.121000e-62,201,0
3990,sp|P75330|P30_MYCPN P30 adhesin OS=Mycoplasma ...,P75330,P75330,100.0,274,274,1.000,1.490000e-162,499,0
3991,sp|Q73U90|OTSB_MYCPA Trehalose-phosphate phosp...,Q73U90,P9WN15,56.8,1327,336,0.859,3.596000e-105,340,144
3992,sp|Q7MU65|LOLD_PORGI Lipoprotein-releasing sys...,Q7MU65,P9WQK1,34.7,248,219,0.982,1.061000e-43,153,140


### Extract Pure And Unique IDs From Headers

In [4]:
headers_list = df['qheader'].tolist()
for i in range(len(headers_list)):
    headers_list[i] = headers_list[i].split(' ')[0]
header_ids = set(headers_list)
print("Unique IDs: ", len(header_ids))

count=0
for i in header_ids:
    print(i)
    count+=1
    if count==10:
        break

Unique IDs:  2421
sp|O06159|BKDC_MYCTU
sp|A0PL16|KDC_MYCUA
sp|P33570|TKT2_ECOLI
sp|Q9CBE5|PSTS3_MYCLE
sp|Q2FXM8|PFKA_STAA8
sp|O07884|FLGE_TREPA
sp|O51768|TOP1_BORBU
sp|P53006|ATPFD_MYCLE
sp|P43741|DPO1_HAEIN
sp|P64281|GREA_SALTY


### Filtering Non-Antigen Sequences And Saving Into A FASTA File

In [5]:
with open("filtered_non_antigens.fasta", "w") as output_file:
    with open("../../SEQUENCES-COLLECTION/non_antigenic_sequences.fasta", "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            if record.id not in header_ids:
                SeqIO.write(record, output_file, "fasta")

### Verifying Whether We Removed All Sequences Identical To Antigens

In [6]:
ids=[]
with open("../../SEQUENCES-COLLECTION/non_antigenic_sequences.fasta", "r")as file:
    for record in SeqIO.parse(file, "fasta"):
        ids.append(record.id)
print("No.of Original sequences:", len(ids))
print("No.of original sequences - No.of identical sequences to antigens:", len(ids), "-", len(header_ids), "=", len(ids) - len(header_ids))

ids=[]
with open("filtered_non_antigens.fasta", "r")as file:
    for record in SeqIO.parse(file, "fasta"):
        ids.append(record.id)
print("No.of Filtered sequences:", len(ids))

No.of Original sequences: 25274
No.of original sequences - No.of identical sequences to antigens: 25274 - 2421 = 22853
No.of Filtered sequences: 22853
